In [ ]:
import cv2
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from skimage import exposure
import random


In [ ]:


# Define the video directory
video_directory = 'C:/Users/Ashis/Desktop/Untitled Folder/videos'


# Functions for reading videos from directory and extracting frames
def read_videos_from_directory(directory):
    videos = []
    for filename in os.listdir(directory):
        if filename.endswith('.mov'):
            videos.append(os.path.join(directory, filename))
    return videos


def extract_frames(video_file, output_directory):
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Error: Failed to open video file: {video_file}")
        return

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (320, 240))
        output_file = os.path.join(output_directory, f'frame_{frame_count}.jpg')
        cv2.imwrite(output_file, frame)
        frame_count += 1
    cap.release()


# Extract HOG features
def extract_hog_features(image):
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True)
    hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
    return fd


# Read videos and extract frames
video_files = read_videos_from_directory(video_directory)
output_directory = 'C:/Users/Ashis/Desktop/Untitled Folder/frames'

print("Video files found:", video_files)

print("Extracting frames from videos...")
for video_file in video_files:
    print(f"Extracting frames from {video_file}...")
    extract_frames(video_file, output_directory)
print("Frames extraction complete")


In [ ]:
# Collect HOG features and labels
X = []
y = []

for root, dirs, files in os.walk(output_directory):
    for file in files:
        if file.endswith(".jpg"):
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img is None:
                print(f"Error: Unable to read image file: {img_path}")
                continue

            # Convert image to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            # Extract HOG features
            hog_features = extract_hog_features(gray)
            X.append(hog_features)

            # Assuming presence of cars is indicated by the directory structure
            if "car" in root:
                y.append(1)
            else:
                y.append(0)


In [ ]:
# Check the unique labels in the dataset
unique_labels = set(y)
if len(unique_labels) < 2:
    print("Error: Dataset must contain samples from at least two classes.")
    print("Unique labels found in the dataset:", unique_labels)
    exit()

In [ ]:
# Step 1: Split the videos into training, testing, and evaluation sets
video_files = read_videos_from_directory(video_directory)
random.shuffle(video_files)  # Shuffle the videos randomly

train_videos = video_files[:20]
test_videos = video_files[20:25]  # 5 videos for testing
eval_videos = video_files[25:]    # 5 videos for evaluation


In [ ]:
# Step 2: Extract frames and collect features for each set
X_train, y_train = collect_features_from_videos(train_videos)
X_test, y_test = collect_features_from_videos(test_videos)
X_eval, y_eval = collect_features_from_videos(eval_videos)


In [ ]:
# Step 3: Train the model
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)

In [ ]:
# Step 4: Test the model
y_test_pred = svm_classifier.predict(X_test)
accuracy_test = accuracy_score(y_test, y_test_pred)
print("Accuracy on the testing set:", accuracy_test)

In [ ]:
# Step 5: Evaluate the model
y_eval_pred = svm_classifier.predict(X_eval)
accuracy_eval = accuracy_score(y_eval, y_eval_pred)
print("Accuracy on the evaluation set:", accuracy_eval)